# Preparing Data for Graph Construction

As part of our larger data science goal for this workshop, we will be working with data reflecting the entire road network of Great Britain. We have as a starting point road data extracted into tabular csv format from official [GML](https://en.wikipedia.org/wiki/Geography_Markup_Language) files. Ultimately, we would like to use cuGraph to perform GPU-accelerated graph analytics on this data, but in order to do so, we need to do some preprocessing to get it ready for graph creation.

In this notebook you will be learning additional cuDF data transformation techniques, including the use of merges, in a demonstration of prepping the initial data for ingestion by cuGraph. Next, you will do a series of exercises to perform a similar transformation of the data for the creation of a graph with different edge weights.

## Objectives

By the time you complete this notebook you will be able to:

- Perform GPU-accelerated merge operations with cuDF
- Prepare data for cuGraph ingest
- Create a GPU-accelerated graph

## Imports

In addition to `cudf`, for this notebook we will also import `cugraph`, which we will use (after data preparation) to construct a GPU-accelerated graph. We also import `networkx` for a brief performance comparison later on.

In [11]:
import cudf
import cugraph as cg

import networkx as nx

## Read Data

In this notebook we will be working with two data sources that will help us create a graph of the UK's road networks.

### UK Road Nodes

The first data table describes the nodes in the road network: endpoints, junctions (including roundabouts), and points that break up a long stretch of curving road so that it can be mapped correctly (instead of as a straight line).

The coordinates for each point are in the OSGB36 format we explored earlier in section 1-05.

In [12]:
road_nodes = cudf.read_csv('./data/road_nodes_1-06.csv')
road_nodes.head()

,node_id,east,north,type
0,id02FE73D4-E88D-4119-8DC2-6E80DE6F6594,320608.0938,870994.0000,junction
1,id634D65C1-C38B-4868-9080-2E1E47F0935C,320628.5000,871103.8125,road end
2,idDC14D4D1-774E-487D-8EDE-60B129E5482C,320635.4688,870983.9375,junction
3,id51555819-1A39-4B41-B0C9-C6D2086D9921,320648.7188,871083.5625,junction
4,id9E362428-79D7-4EE3-B015-0CE3F6A78A69,320658.1875,871162.3750,junction


In [13]:
road_nodes.dtypes

node_id     object
east       float64
north      float64
type        object
dtype: object

In [14]:
road_nodes.shape

(3121148, 4)

In [15]:
road_nodes['type'].unique()

0       junction
1    pseudo node
2       road end
3     roundabout
Name: type, dtype: object

### UK Road Edges

The second data table describes road segments, including their start and end points, how long they are, and what kind of road they are.

In [16]:
road_edges = cudf.read_csv('./data/road_edges_1-06.csv')
road_edges.head()

,src_id,dst_id,length,type,form
0,#id138447A5-91D4-4642-BFAC-13F309705429,#id84C9DAD4-9243-4742-B582-E8CBC848E08A,314,Restricted Local Access Road,Single Carriageway
1,#idD615F9C5-5BE9-412D-9FED-F4928BAB4146,#idA1BB20B9-0751-4B42-9925-20607ABF5027,104,Restricted Local Access Road,Single Carriageway
2,#idDC14D4D1-774E-487D-8EDE-60B129E5482C,#id51555819-1A39-4B41-B0C9-C6D2086D9921,100,Restricted Local Access Road,Single Carriageway
3,#id626FC567-199C-41FB-9F29-1AB718874128,#idACD1B0A9-F870-4B46-88CF-C870A9EDAF8B,93,Restricted Local Access Road,Single Carriageway
4,#id03312900-B147-4CA3-A858-E2BF6AD1ECA7,#id02FE73D4-E88D-4119-8DC2-6E80DE6F6594,95,Restricted Local Access Road,Single Carriageway


In [17]:
road_edges.dtypes

src_id    object
dst_id    object
length     int64
type      object
form      object
dtype: object

In [18]:
road_edges.shape

(3725531, 5)

In [19]:
road_edges['type'].unique()

0                          A Road
1                          B Road
2               Local Access Road
3                      Local Road
4                      Minor Road
5                        Motorway
6    Restricted Local Access Road
7           Secondary Access Road
Name: type, dtype: object

In [20]:
road_edges['form'].unique()

0    Collapsed Dual Carriageway
1              Dual Carriageway
2                 Guided Busway
3                    Roundabout
4        Shared Use Carriageway
5            Single Carriageway
6                     Slip Road
Name: form, dtype: object

## Exercise: Make IDs Compatible

Our csv files were derived from original [GML](https://en.wikipedia.org/wiki/Geography_Markup_Language) files, and as you can see from the above, both `road_edges['src_id']` and `road_edges['dst_id']` contain a leading `#` character that `road_nodes['node_id']` does not. To make the IDs compatible between the edges and nodes, use cuDF's [string method](https://docs.rapids.ai/api/nvstrings/stable/) `.str.lstrip` to replace the `src_id` and `dst_id` columns in `road_edges` with values stripped of the leading `#` characters.

In [21]:
road_edges['src_id'] = road_edges['src_id'].str.lstrip('#')
road_edges['dst_id'] = road_edges['dst_id'].str.lstrip('#')

### Solution

In [22]:
# %load solutions/make_ids_compatible
road_edges['src_id'] = road_edges['src_id'].str.lstrip('#')
road_edges['dst_id'] = road_edges['dst_id'].str.lstrip('#')
road_edges[['src_id', 'dst_id']].head()


,src_id,dst_id
0,id138447A5-91D4-4642-BFAC-13F309705429,id84C9DAD4-9243-4742-B582-E8CBC848E08A
1,idD615F9C5-5BE9-412D-9FED-F4928BAB4146,idA1BB20B9-0751-4B42-9925-20607ABF5027
2,idDC14D4D1-774E-487D-8EDE-60B129E5482C,id51555819-1A39-4B41-B0C9-C6D2086D9921
3,id626FC567-199C-41FB-9F29-1AB718874128,idACD1B0A9-F870-4B46-88CF-C870A9EDAF8B
4,id03312900-B147-4CA3-A858-E2BF6AD1ECA7,id02FE73D4-E88D-4119-8DC2-6E80DE6F6594


## Exercise: Remove Duplicates

Our data originally came from multiple map tiles, and roads which crossed a tile boundary were captured in both the tile from which they started and the tile on which they ended.

Use the `drop_duplicates` method for both `road_nodes` and `road_edges` to remove these duplicate values, and `reset_index` with the parameter `drop=True` to give the new dataframes a contiguous range index.

In [23]:
road_nodes.drop_duplicates()
road_edges.drop_duplicates()
road_edges.reset_index(drop=True)
road_nodes.reset_index(drop=True)

,node_id,east,north,type
0,id02FE73D4-E88D-4119-8DC2-6E80DE6F6594,320608.0938,870994.0000,junction
1,id634D65C1-C38B-4868-9080-2E1E47F0935C,320628.5000,871103.8125,road end
2,idDC14D4D1-774E-487D-8EDE-60B129E5482C,320635.4688,870983.9375,junction
3,id51555819-1A39-4B41-B0C9-C6D2086D9921,320648.7188,871083.5625,junction
4,id9E362428-79D7-4EE3-B015-0CE3F6A78A69,320658.1875,871162.3750,junction
...,...,...,...,...
3121143,id822E4253-1FC9-41F5-BB21-18BFA2C7B236,299267.9063,508339.3125,road end
3121144,id5655B2AF-E3F8-414B-93D5-CBDD617F20D2,299283.9375,508378.6875,road end
3121145,idB8647015-DD34-416A-B195-3F02B096D53F,299361.5000,509276.8125,junction
3121146,id3A811ECE-8672-402E-AD59-C9A381FD49BC,299538.0000,509220.0000,junction


#### Solution

In [24]:
# %load solutions/remove_duplicates
road_nodes = road_nodes.drop_duplicates().reset_index(drop=True)
road_edges = road_edges.drop_duplicates().reset_index(drop=True)


## Data Summary

Now that the data is cleaned we can see just how many roads and endpoints/junctions/curve points we will be working with, as well as its memory footprint in our GPU.

In [25]:
print(f'{road_edges.shape[0]} edges, {road_nodes.shape[0]} nodes')

3667200 edges, 3078117 nodes


In [26]:
!nvidia-smi

Mon Oct  5 01:32:24 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:1B.0 Off |                    0 |
| N/A   47C    P0    59W / 300W |   3556MiB / 16160MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  Off  | 00000000:00:1C.0 Off |                    0 |
| N/A   42C    P0    39W / 300W |     11MiB / 16160MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

## Prep Data for Building the Road Network Graph

The cuGraph graph constructor that we will be using later expects edge sources, edge destinations, and (optionally) edge weights. Edge sources and destinations must be a compact interval of 32-bit integers from 0 to *N*-1, where *N* is the number of nodes in the graph.

Furthermore, in RAPIDS v0.12, undirected edges in cuGraph must be stored as directed edges in both directions. Since our data does not indicate one-way roads, we will assume that all roads can be treated as two-way (undirected).

These constraints will require some preparation, but they allow for the blazing-fast analytics we will be exploring later.

As it stands, and contrary to our needs, our `road_edges` dataframe consists of `src_id` and `dst_id` strings which match `node_id` values in the `road_nodes`. Furthermore, our road edges are recorded as going from one source to one destination even when they are two-way roads.

In [27]:
road_edges[['src_id', 'dst_id']].dtypes

src_id    object
dst_id    object
dtype: object

In [28]:
road_edges['src_id'][0]

'id000000F5-5180-4C03-B05D-B01352C54F89'

In [29]:
road_edges['dst_id'][0]

'id0ABF4CB2-6997-4952-A9AA-313DE2EF6538'

In [30]:
road_nodes['node_id'].dtype

dtype('O')

In [31]:
road_nodes['node_id'][0]

'id000000F5-5180-4C03-B05D-B01352C54F89'

In order to prepare our data for constructing a graph with cuGraph, we will do the following:

1. Create a unique int32 value for each `node_id`, which we will call that node's `graph_id`, and save the mapping from `node_id` to `graph_id`. Whenever we get results from analyzing the graph, we can use the mapping to look up which original node corresponding to that `graph_id`.
2. Use the mapping to update `road_edges` with `graph_id`s for each edge's start and end points. We will ultimately call the mapped source points `graph_src` and the destination points `graph_dst`.
3. Create a new dataframe with the same edges, but with swapped source and destination points, then concatenate that swapped dataframe to the original and deduplicate (so that each road will be treated in the graph as two-way).

### Create `graph_id`s

The dataframe method `factorize` produces both an integer labeling of the `node_id` values and the mapping series that reflects that labeling.

In [32]:
all_node_ids = road_edges['src_id'].append(road_edges['dst_id'], ignore_index=True) # ignore_index to prevent index overlap
graph_ids, mapping_series = all_node_ids.factorize()
mapping_series.head()

0    id000000F5-5180-4C03-B05D-B01352C54F89
1    id000003F8-9E09-4829-AD87-6DA4438D22D8
2    id000010DA-C89A-4198-847A-6E62815E038A
3    id000017A0-1843-4BC7-BCF7-C943B6780839
4    id00001B2A-155F-4CD3-8E06-7677ADC6AF74
dtype: object

### Reattach `graph_id`s onto `road_edges`

We now need to add `graph_id` columns into `road_edges` twice: once to create a new source ID column, and again to create a new destination ID column. Because we preserved the order of the `src_id` and `dst_id` columns when sending them to `factorize`, we can simply split the resulting `graph_id`s in half and attach them back to `road_edges`.

In [33]:
half_index = int((all_node_ids.shape[0]/2))
road_edges['graph_src'] = graph_ids.iloc[:half_index]
road_edges['graph_dst'] = graph_ids.iloc[half_index:].reset_index(drop=True) # reset_index to reverse the ignore_index above
road_edges.head()

,src_id,dst_id,length,type,form,graph_src,graph_dst
0,id000000F5-5180-4C03-B05D-B01352C54F89,id0ABF4CB2-6997-4952-A9AA-313DE2EF6538,44,Local Road,Single Carriageway,0,129165
1,id000003F8-9E09-4829-AD87-6DA4438D22D8,id8B9308E8-DD5C-4C5B-AE92-A8F6C471B551,70,Secondary Access Road,Single Carriageway,1,1678323
2,id000003F8-9E09-4829-AD87-6DA4438D22D8,idCE926C8D-A698-410F-BE88-0EDE56BB0476,40,Local Road,Single Carriageway,1,2483057
3,id000010DA-C89A-4198-847A-6E62815E038A,id000010DA-C89A-4198-847A-6E62815E038A,55,Local Road,Single Carriageway,2,2
4,id000017A0-1843-4BC7-BCF7-C943B6780839,idECFC5CB9-4FD9-4377-8D68-00B1D2654547,71,Local Road,Single Carriageway,3,2848890


### Merge `graph_id` into `road_nodes`

cuDF provides merging functionality just like Pandas. Even though we don't directly need `road_nodes` to construct our graph, we will need it later in the workshop to geospatially analyze where the roads are.

Since we will join the mapping to `road_nodes` via the `node_id` column, we convert the mapping's index to a `graph_id` column.

In [34]:
node_id_to_graph_id_mapping = cudf.DataFrame()
node_id_to_graph_id_mapping['node_id'] = mapping_series
node_id_to_graph_id_mapping['graph_id'] = cudf.Series(mapping_series.index)
node_id_to_graph_id_mapping.head()

,node_id,graph_id
0,id000000F5-5180-4C03-B05D-B01352C54F89,0
1,id000003F8-9E09-4829-AD87-6DA4438D22D8,1
2,id000010DA-C89A-4198-847A-6E62815E038A,2
3,id000017A0-1843-4BC7-BCF7-C943B6780839,3
4,id00001B2A-155F-4CD3-8E06-7677ADC6AF74,4


We now add the `graph_id` column to `road_nodes` by performing a merge:

In [35]:
%time road_nodes = road_nodes.merge(node_id_to_graph_id_mapping, on='node_id', how='left')
road_nodes.head()

CPU times: user 88 ms, sys: 104 ms, total: 192 ms
Wall time: 188 ms


,node_id,east,north,type,graph_id
0,id040D1AA6-DECA-4E49-8B79-077932272CBB,450737.0000,506461.000,road end,48736
1,id040D1BFB-B7A3-45DB-B7B3-B547CC135AEA,506833.0000,102916.000,junction,48737
2,id040D26F4-D78C-4077-9DA6-5FC560C6A5D6,433340.0000,416171.000,road end,48738
3,id040D3244-20A1-447C-98BC-AED626CCF1BE,413415.3750,429897.625,junction,48739
4,id040D3D22-832F-49EF-A8A8-6F53188CC87F,241705.2969,599384.375,junction,48740


#### Reindex `road_nodes`

For efficient lookup later, we will reindex `road_nodes` to use the `graph_id` as its index - remember, we will typically get results from the graph analytics in terms of `graph_id`s, so this lets us easily pull other information about the nodes (like their locations). We then sort the dataframe on this new index:

In [36]:
road_nodes = road_nodes.set_index('graph_id', drop=True)
%time road_nodes = road_nodes.sort_index()
road_nodes.head()

CPU times: user 48 ms, sys: 72 ms, total: 120 ms
Wall time: 120 ms


,node_id,east,north,type
graph_id,,,,
0,id000000F5-5180-4C03-B05D-B01352C54F89,432920.250,572547.4375,road end
1,id000003F8-9E09-4829-AD87-6DA4438D22D8,526616.375,189678.3906,junction
2,id000010DA-C89A-4198-847A-6E62815E038A,336879.000,731824.0000,junction
3,id000017A0-1843-4BC7-BCF7-C943B6780839,380635.000,390153.0000,junction
4,id00001B2A-155F-4CD3-8E06-7677ADC6AF74,337481.000,350509.7188,junction


### Build Dataframe with Columns Needed for Graph Construction

Before making our graph data undirected, let's construct a smaller dataframe with only the columns we will need to give to the cuGraph constructor so we can avoid repeatedly moving unnecessary data:

In [37]:
graph_prep = cudf.DataFrame()
graph_prep['src']    = road_edges['graph_src']
graph_prep['dst']    = road_edges['graph_dst']
graph_prep['length'] = road_edges['length'].astype('float32') # cuGraph expects edge weights as floating points
print(graph_prep.shape)
graph_prep.dtypes

(3667200, 3)


src         int32
dst         int32
length    float32
dtype: object

### Make Graph Undirected

We now make our graph undirected by creating a new dataframe with sources and destinations swapped, concatenating that new dataframe to `graph_prep`:

In [38]:
rev_gdf = cudf.DataFrame()

rev_gdf['src'] = graph_prep['dst']
rev_gdf['dst'] = graph_prep['src']
rev_gdf['length'] = graph_prep['length']

graph_prep = cudf.concat([graph_prep, rev_gdf], 
                          ignore_index=True)
graph_prep.shape

(7334400, 3)

While it's possible to have road edges with the same starting and ending nodes but different lengths (say, a loop that breaks off from and rejoins a main road), for simplicity we will deduplicate the edge list while ignoring `length` so that there is only one (undirected) edge between each pair of connected nodes.

In [39]:
graph_prep.drop_duplicates(subset=['src', 'dst'], inplace=True)
graph_prep.shape

(7218169, 3)

Let's confirm `graph_prep` contains a compact interval of int32 IDs:

In [43]:
# The min src or dst ID is 0
graph_prep[['src', 'dst']].min().min() == 0

True

In [41]:
# The max src or dst id is the number of nodes - 1
graph_prep[['src', 'dst']].max().max() == road_nodes['node_id'].unique().shape[0] - 1

True

In [42]:
# The ID dtypes are int32s
graph_prep[['src', 'dst']].dtypes

src    int32
dst    int32
dtype: object

## Construct Graph with cuGraph

Now that we have `graph_prep` prepped correctly, we can use cuGraph to create our graph, which we can then use for accelerated analysis. In order to do so, we first use cuGraph to instantiate a `Graph` instance, and then initialize it with edge sources, edge destinations, and edge weights, which for our data will be the length of the roads:

In [44]:
G = cg.Graph()
%time G.from_cudf_edgelist(graph_prep, source='src', destination='dst', edge_attr='length')

CPU times: user 704 ms, sys: 72 ms, total: 776 ms
Wall time: 773 ms


You can see how fast the whole preparation and construction process can be, even starting from the beginning, by restarting the kernel, clicking on this text, and then selecting "Run All Above Selected Cell" from the Run menu (should take under ten seconds).

Just as a point of comparison, we also construct the equivalent graph in NetworkX from the equivalent cleaned and prepped Pandas dataframe. This should take a minute or so - feel free to read ahead while it processes.

In [45]:
graph_prep_cpu = graph_prep.to_pandas()

%time G_cpu = nx.convert_matrix.from_pandas_edgelist(graph_prep_cpu, source='src', target='dst', edge_attr='length')

CPU times: user 47.6 s, sys: 1.1 s, total: 48.7 s
Wall time: 48.7 s


### Analyzing the Graph

Now that we have created the graph we can analyze the number of nodes and edges in it:

In [46]:
G.number_of_nodes()

3078117

In [47]:
G.number_of_edges()

7218169

We can also analyze the degrees of our graph nodes:

In [48]:
deg_df = G.degree()

We duplicated every edge to make the graph undirected, so we expect the nodes to have a minimum degree of 2:

In [49]:
deg_df['degree'].describe()[1:]

mean     4.689990
std      1.913452
min      2.000000
25%      2.000000
50%      6.000000
75%      6.000000
max     16.000000
Name: degree, dtype: float64

You will spend more time using this GPU-accelerated graph later in the workshop.

## Exercise: Construct a Graph of Roads with Time Weights

For this series of exercises, you are going to construct and analyze a new graph of Great Britain's roads using the techniques just demonstrated, but this time, instead of using raw distance for the edges' weights, you will be using the time it will take to travel between the two nodes.

You will be beginning this exercise with the `road_edges` dataframe from earlier, which already contains the unique int32 `graph_id`s we need for graph construction in `graph_src` and `graph_dst`:

In [50]:
road_edges.dtypes

src_id       object
dst_id       object
length        int64
type         object
form         object
graph_src     int32
graph_dst     int32
dtype: object

### Road Type to Speed Conversion

In order to calculate how long it should take to travel along a road, we need to know its speed limit. We will do this by utilizing `road_edges['type']`, along with rules for the speed limits for each type of road.

Here are the unique types of roads in our data:

In [51]:
road_edges['type'].unique()

0                          A Road
1                          B Road
2               Local Access Road
3                      Local Road
4                      Minor Road
5                        Motorway
6    Restricted Local Access Road
7           Secondary Access Road
Name: type, dtype: object

And here is a table with assumptions about speed limits we can use for our conversion:

In [52]:
# https://www.rac.co.uk/drive/advice/legal/speed-limits/
# Technically, speed limits depend on whether a road is in a built-up area and the form of carriageway,
# but we can use road type as a proxy for built-up areas.
# Values are in mph.

speed_limits = {'Motorway': 70,
               'A Road': 60,
               'B Road': 60,
               'Local Road': 30,
               'Local Access Road': 30,
               'Minor Road': 30,
               'Restricted Local Access Road': 30,
               'Secondary Access Road': 30}

We begin by creating `speed_gdf` to store each road type with its speed limit:

In [53]:
speed_gdf = cudf.DataFrame()

speed_gdf['type'] = speed_limits.keys()
speed_gdf['limit_mph'] = [speed_limits[key] for key in speed_limits.keys()]
speed_gdf

,type,limit_mph
0,Motorway,70
1,A Road,60
2,B Road,60
3,Local Road,30
4,Local Access Road,30
5,Minor Road,30
6,Restricted Local Access Road,30
7,Secondary Access Road,30


Next we add an additional column, `limit_m/s`, which for each road type will give us a measure of how fast one can travel on it in meters / second.

In [54]:
# We will have road distances in meters (m), so to get road distances in seconds (s), we need to multiply by meters/mile and divide by seconds/hour
# 1 mile ~ 1609.34 m
speed_gdf['limit_m/s'] = speed_gdf['limit_mph'] * 1609.34 / 3600
speed_gdf

,type,limit_mph,limit_m/s
0,Motorway,70,31.292722
1,A Road,60,26.822333
2,B Road,60,26.822333
3,Local Road,30,13.411167
4,Local Access Road,30,13.411167
5,Minor Road,30,13.411167
6,Restricted Local Access Road,30,13.411167
7,Secondary Access Road,30,13.411167


### Step 1: Merge `speed_gdf` into `road_edges`

Since we will be using values in `road_edges` to construct our graph, we need to merge `speed_gdf` into `road_edges`. You can join on the `type` column, which both of these dataframes share.

In [59]:
%time road_edges = road_edges.merge(speed_gdf, on='type')

CPU times: user 180 ms, sys: 196 ms, total: 376 ms
Wall time: 376 ms


#### Solution

In [56]:
# %load solutions/merge_speed_gdf
road_edges = road_edges.merge(speed_gdf, on='type')
road_edges.dtypes


### Step 2: Add Length in Seconds Column

You now need to calculate the number of seconds it will take to traverse a given road at the speed limit. This can be done by dividing a road's length in m by its speed limit in m/s. Perform this calculation on `road_edges` and store the results in a new column `length_s`.

In [61]:
road_edges['length_s'] = road_edges['length'] / road_edges['limit_m/s']
road_edges.head()

,src_id,dst_id,length,type,form,graph_src,graph_dst,limit_mph,limit_m/s,length_s
0,id002F39E2-6647-4BEF-BB9E-D96456E54696,idC04BE207-03A6-40E6-8C05-0E645B4AA50B,47,Local Road,Single Carriageway,2254,2311469,30,13.411167,3.504542
1,id002F541E-911C-4790-A40A-A0E2928947F3,id432C170D-E65B-48DF-AF5E-16FF3764697D,247,Minor Road,Single Carriageway,2257,807013,30,13.411167,18.417488
2,id002F55C3-C66D-483E-83C2-38A9A0292281,id514D101B-DDE8-4448-A30C-91A6BC477C63,125,Local Road,Single Carriageway,2258,977111,30,13.411167,9.320591
3,id002F55C3-C66D-483E-83C2-38A9A0292281,idF80E27D9-D6A7-4492-A447-4CE2ACD59DAD,33,Local Road,Single Carriageway,2258,2982287,30,13.411167,2.460636
4,id002F594C-D9D0-490F-A9DD-E432F862F84E,id77861574-440D-494C-8C3C-4E63B353C641,21,Local Road,Single Carriageway,2259,1437088,30,13.411167,1.565859


#### Solution

In [62]:
# %load solutions/length_in_seconds
road_edges['length_s'] = road_edges['length'] / road_edges['limit_m/s']
road_edges['length_s'].head()


### Step 3: Construct a Prep Dataframe

Make a new dataframe called `exercise_graph`, add to it three columns: `src`, `dst`, and `length_s`, all which you can obtain from `road_edges`.

In [67]:
exercise_graph = cudf.DataFrame()
exercise_graph['src'] = road_edges['graph_src']
exercise_graph['dst'] = road_edges['graph_dst']
exercise_graph['length_s'] = road_edges['length_s']

#### Solution

In [64]:
# %load solutions/prep_dataframe
exercise_graph = cudf.DataFrame()

exercise_graph['src'] = road_edges['graph_src']
exercise_graph['dst'] = road_edges['graph_dst']
exercise_graph['length_s'] = road_edges['length_s']

print(exercise_graph.shape)
exercise_graph.dtypes


### Step 4: Make the Edges Undirected

Make `exercise_graph` undirected by creating a dataframe where `src` and `dst` are reversed, and then concatenating it with `exercise_graph`. Be sure to drop any rows that duplicate other rows' `src` and `dst` values.

In [71]:
rev_exercise_graph = cudf.DataFrame()

rev_exercise_graph['src'] = exercise_graph['dst']
rev_exercise_graph['dst'] = exercise_graph['src']
rev_exercise_graph['length_s'] = exercise_graph['length_s']

exercise_graph = cudf.concat([exercise_graph, rev_exercise_graph], 
                          ignore_index=True)
exercise_graph.drop_duplicates(subset=['src', 'dst'])


,src,dst,length_s
5952,0,129165,3.280848
5953,1,1678323,5.219531
6490552,1,2372610,1.342165
5954,1,2483057,2.982589
5955,2,2,4.101060
...,...,...,...
3663581,3078114,3057280,50.107497
6917713,3078115,2721612,7.009084
5720304,3078116,1721226,5.666919
3663582,3078116,2979684,10.289933


#### Solution

In [73]:
# %load solutions/make_undirected
rev_gdf = cudf.DataFrame()

rev_gdf['src'] = exercise_graph['dst']
rev_gdf['dst'] = exercise_graph['src']
rev_gdf['length_s'] = exercise_graph['length_s']

exercise_graph = cudf.concat([exercise_graph, rev_gdf], 
                              ignore_index=True)

print(exercise_graph.shape)

exercise_graph.drop_duplicates(subset=['src', 'dst'], inplace=True)
print(exercise_graph.shape)

# The maximum graph_id is the number of nodes - 1
print(exercise_graph[['src', 'dst']].max().max() == road_nodes['node_id'].unique().shape[0] - 1)

# The minimum graph_id is 0 
print(exercise_graph[['src', 'dst']].min().min() == 0)

# The ID dtypes are int32s
print(exercise_graph[['src', 'dst']].dtypes == 'int32')


(29337600, 3)
(7218169, 3)
False
True
src    True
dst    True
dtype: bool


### Step 5: Construct the Graph

Construct a cuGraph Graph called `G_ex` using the sources and destinations found in `exercise_graph`, along with length in seconds values for the edges' weights.

In [72]:
G_ex = cg.Graph()
%time G_ex.from_cudf_edgelist(exercise_graph, source='src', destination='dst', edge_attr='length_s')

CPU times: user 84 ms, sys: 48 ms, total: 132 ms
Wall time: 134 ms


#### Solution

In [74]:
# %load solutions/construct_graph
G_ex = cg.Graph()
G_ex.add_edge_list(exercise_graph['src'], exercise_graph['dst'], exercise_graph['length_s'])


<br>
<div align="center"><h2>Please Restart the Kernel</h2></div>

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

## Next

In the next notebook you will work with a dataset representing a population 5 times larger than the UK, a dataset that would not fit in the memory of a single GPU. In order to work with this data you will use Dask cuDF to partition the data among the 4 GPUs at your disposal, and perform the same kinds of data manipulations you have been doing with vanilla cuDF on smaller, single-GPU datasets.